# Final code- 04 Dec

In [26]:
import random

# Number of Players, Rounds, and Simulations
num_players = 10
num_rounds = 5
num_simulations = 1000
k_factor = 10  # Higher K-factor for more responsive Elo rating updates

# Initialize Players with Elo ratings
players = [{'name': f'Player {i}', 'elo': random.randint(1200, 2000), 'points': 0, 'wins': 0, 'current_form': 0} for i in range(num_players)]

for player in players:
    print(f"{player ['name']} - Init Elo: {player['elo']:.2f}")
    
def calculate_win_probability(player1, player2):
    """ Calculate the probability of player1 winning against player2, considering current form """
    # Adjusting Elo ratings based on current form
    adjusted_elo1 = player1['elo'] + player1['current_form']
    adjusted_elo2 = player2['elo'] + player2['current_form']
    return 1 / (1 + 10 ** ((adjusted_elo2 - adjusted_elo1) / 400))

def update_elo_ratings(winner, loser, draw=False):
    """ Update Elo ratings and current form after a game """
    expected_win_winner = calculate_win_probability(winner, loser)
    expected_win_loser = calculate_win_probability(loser, winner)

    score_winner = 0.5 if draw else 1
    score_loser = 0.5 if draw else 0

    winner['elo'] += k_factor * (score_winner - expected_win_winner)
    loser['elo'] += k_factor * (score_loser - expected_win_loser)

    # Update current form based on recent game outcome
    winner['current_form'] += k_factor * 1.2 if not draw else -k_factor * 0.3
    loser['current_form'] += -k_factor * 1.2 if not draw else k_factor * 0.5

def simulate_game(player1, player2):
    """ Simulate a game between two players """
    win_prob = calculate_win_probability(player1, player2)
    result = random.random()
    if result < win_prob:
        player1['points'] += 1  # Player 1 wins
        update_elo_ratings(player1, player2)
    elif result < win_prob + (1 - win_prob) / 2:
#         print(player1['name'], player2['name'], " draw")
        player1['points'] += 0.5
        player2['points'] += 0.5  # Draw
        update_elo_ratings(player1, player2, draw=True)
    else:
        player2['points'] += 1  # Player 2 wins
        update_elo_ratings(player2, player1)

def swiss_pairing(players):
    """ Pair players using a simple Swiss system approach """
    sorted_players = sorted(players, key=lambda x: x['points'], reverse=True)
    pairings = []
    while sorted_players:
        player1 = sorted_players.pop(0)
        for i, player2 in enumerate(sorted_players):
            if not 'opponents' in player1 or player2['name'] not in player1['opponents']:
                player1.setdefault('opponents', []).append(player2['name'])
                player2.setdefault('opponents', []).append(player1['name'])
                pairings.append((player1, player2))
                sorted_players.pop(i)
                break
    return pairings

def print_final_round_statistics(players, simulation_number):
    """ Print statistics for the final round """
    if 9990 <= simulation_number <= 10000:
        print(f"\nSimulation {simulation_number} - Final Round Results:")
        for player in players:
            winning_probability = player['wins'] / num_simulations
            print(f"{player['name']} - Points: {player['points']}, Elo: {player['elo']:.2f}, Winning Probability: {winning_probability:.2f}")

def simulate_tournament():
    """ Simulate a single Swiss-system tournament """
    # Reset points and opponents for each player
    for player in players:
        player['points'] = 0
        player['opponents'] = []
        player['current_form'] = 0

    # Simulate rounds
    for round_number in range(1, num_rounds + 1):
        pairings = swiss_pairing(players)
        for player1, player2 in pairings:
            simulate_game(player1, player2)

    # Determine winner and update wins
    winner = max(players, key=lambda x: x['points'])
    winner['wins'] += 1
    
    
# Reset wins count before all simulations
for player in players:
    player['wins'] = 0

# Simulate all tournaments
for _ in range(1, num_simulations + 1):
    simulate_tournament()

    
# Calculate and print final probabilities after all simulations
print("\nFinal Tournament Results after all simulations:")
for player in players:
    winning_probability = player['wins'] / num_simulations
    print(f"{player['name']} - - Points: {player['points']} - - Final Elo: {player['elo']:.2f} - - Winning Probability: {winning_probability:.2f}")


Player 0 - Init Elo: 1556.00
Player 1 - Init Elo: 1548.00
Player 2 - Init Elo: 1991.00
Player 3 - Init Elo: 1214.00
Player 4 - Init Elo: 1232.00
Player 5 - Init Elo: 1698.00
Player 6 - Init Elo: 1396.00
Player 7 - Init Elo: 1531.00
Player 8 - Init Elo: 1751.00
Player 9 - Init Elo: 1707.00

Final Tournament Results after all simulations:
Player 0 - - Points: 4.5 - - Final Elo: 2165.36 - - Winning Probability: 0.76
Player 1 - - Points: 1 - - Final Elo: 1236.82 - - Winning Probability: 0.01
Player 2 - - Points: 2.5 - - Final Elo: 1606.58 - - Winning Probability: 0.12
Player 3 - - Points: 3 - - Final Elo: 1609.79 - - Winning Probability: 0.00
Player 4 - - Points: 4.5 - - Final Elo: 1989.47 - - Winning Probability: 0.05
Player 5 - - Points: 1 - - Final Elo: 1345.61 - - Winning Probability: 0.01
Player 6 - - Points: 2 - - Final Elo: 1662.80 - - Winning Probability: 0.01
Player 7 - - Points: 2.5 - - Final Elo: 1485.15 - - Winning Probability: 0.00
Player 8 - - Points: 3 - - Final Elo: 1722.38